In [ ]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  %pip install --quiet git+https://github.com/raultruco/cryptowatson-indicators#egg=cryptowatson-indicators
from cryptowatsonindicators.rwa import Rwa
import requests
requests.packages.urllib3.disable_warnings()
import pprint
pprint = pprint.PrettyPrinter(indent=2, sort_dicts=False, compact=False).pprint   # pprint with defaults

### Global variables

In [ ]:
ticker_symbol = "BTCUSDT"    # currently only works with BTCUSDT
start_date = None            # Limt time series to start at specific date or None to use all the history
binance_api_key = ''         # use your binance keys or leave blank to use Binance's testnet
binance_secret_key = ''
get_current_band = True
get_band_at = True
plot_rainbow_chart = True
run_simulator = True

rwa = Rwa(start_date=start_date, ticker_symbol=ticker_symbol, binance_api_key=binance_api_key, binance_secret_key=binance_secret_key)

### Get rainbow band

In [ ]:
if get_current_band:
    rainbow_band = rwa.get_current_rainbow_band()
    print('Current Rainbow Band:')
    pprint(rainbow_band)

In [ ]:
if get_band_at:
    at_date = '01/04/2022'    # date when look up the band
    price_at_date = 30000     # Price of BTC at date
    rainbow_band = rwa.get_rainbow_band(price=price_at_date, at_date=at_date)
    print(f"Rainbow Band at {at_date}:")
    pprint(rainbow_band)

### Plot Rainbow chart

In [ ]:
if plot_rainbow_chart:
    rwa.plot_rainbow()

### Simulator

Usage...

In [ ]:

start_date = '01/01/2020'    # start date of the simulation
buy_frequency = 7            # Number of days between purchases
buy_amount = 100             # Amount purchased in standard DCA
weight_type = "fibs"         # "fibs" or "originaldca"

if run_simulator:
    pass
